<a href="https://colab.research.google.com/github/laurensg420/SMR2/blob/master/train_tf_2_0_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Train with GPU:

In [1]:
import os, sys, math
import numpy as np
if 'google.colab' in sys.modules: # Colab-only Tensorflow version selector
  %tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


TensorFlow 2.x selected.
Tensorflow version 2.1.0-rc1
Found GPU at: /device:GPU:0


In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
0.16974323099999822
GPU (s):
0.1652511309999909
GPU speedup over CPU: 1x


In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os

base_path = '/content/drive/My Drive/ML Mofos SMR/colab_files/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# test writing file on Google Drive
with open('/content/drive/My Drive/ML Mofos SMR/colab_files/test.txt', 'w+') as f:
  f.write('Hello Google Drive!')
!cat /content/drive/My\ Drive/ML\ Mofos\ SMR/colab_files/test.txt

Hello Google Drive!

In [0]:
!cp /content/drive/My\ Drive/ML\ Mofos\ SMR/colab_files/squeezenet.py /content/squeezenet.py


In [0]:
!mkdir -p /content/dataset/image_data_blue_light_split && cp -r /content/drive/My\ Drive/ML\ Mofos\ SMR/colab_files/dataset/image_data_blue_light_split /content/dataset/

In [0]:
!mkdir -p /content/models/
base_path = '/content/'


In [5]:
from squeezenet import SqueezeNet, SqueezeNet_11
import os, sys, math
import numpy as np
if 'google.colab' in sys.modules: # Colab-only Tensorflow version selector
  %tensorflow_version 1.x
import tensorflow
print("Tensorflow version " + tensorflow.__version__)

import cv2.cv2 as cv2
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation, Dropout, Convolution2D, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger

import os

train_datagen = ImageDataGenerator(
    # featurewise_center=False,
    # samplewise_center=False,
    # featurewise_std_normalization=False,
    # samplewise_std_normalization=False,
    # zca_whitening=False,
    # zca_epsilon=1e-06,
    # rotation_range=0,
    # width_shift_range=0.0,
    # height_shift_range=0.0,
    brightness_range=(0.8,1.2), # doc on brightness_range = https://pillow.readthedocs.io/en/3.0.x/reference/ImageEnhance.html#PIL.ImageEnhance.Brightness
    # shear_range=0.0,
    # zoom_range=0.0,
    # channel_shift_range=0.0,
    # fill_mode='nearest',
    # cval=0.0,
    # rotation_range=20,
    horizontal_flip=True,
    # vertical_flip=False,
    # rescale=None,
    # preprocessing_function=None,
    # data_format='channels_last',
    # validation_split=0.0,
    # interpolation_order=1,
    # dtype='float32'
    )

# settings
train_batch_size = 160
val_batch_size = 160
roi_square_size = 350


# train_datagen = ImageDataGenerator(
# rescale=1./255,
# shear_range=0.2,
# zoom_range=0.2,
# horizontal_flip=True)

test_datagen = ImageDataGenerator()
train_it = train_datagen.flow_from_directory(
    os.path.join( base_path,'dataset/image_data_blue_light_split/train' ),
    target_size=(roi_square_size, roi_square_size),
    class_mode='categorical',
    batch_size=train_batch_size
)

val_it = test_datagen.flow_from_directory(
    os.path.join( base_path,'dataset/image_data_blue_light_split/validate' ),
    target_size=(roi_square_size, roi_square_size),
    class_mode='categorical',
    batch_size=val_batch_size
)

amount_classes = len( os.listdir( os.path.join( base_path, 'dataset/image_data_blue_light_split/train' ) ) )


amount_train_images = train_it.samples
amount_val_images = val_it.samples


batches_in_dataset = (amount_train_images / train_batch_size)
epochs = int( batches_in_dataset ) * 10

model_name = os.path.join( base_path, 'models/image_data_blue_light_split'\
       + '_sq' + str(roi_square_size)\
       + '_e'  + str(epochs)\
       + '_tb' + str(train_batch_size)\
       + '_vb' + str(val_batch_size)\
       + '_aug-hor-rotran20-briran0_8;1_2)'\
       +'.h5')

print( "batches_in_dataset:", batches_in_dataset)
print( "epochs:", epochs )
print( "train_batch_size:", train_batch_size)
print( "val_batch_size:", val_batch_size )
print( "model_name:", model_name)

def get_model(amount_classes):
    model = SqueezeNet_11( input_shape=(roi_square_size, roi_square_size, 3), nb_classes=amount_classes)
    return model


# define the model
model = get_model( amount_classes )
model.compile(
    optimizer=Adam( lr=0.0001 ),
    loss='categorical_crossentropy',
    metrics=['acc']#,tf.metrics.TruePositives(),tf.metrics.FalsePositives()]
)  # lr = learning rate

# save checkpoints after epochs
filepath=os.path.join(base_path,"weights-improvement-e{epoch:02d}-va{val_acc:.2f}.h5")
print(filepath)
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='max', save_freq=5)

csv_logger = CSVLogger(model_name.split('.')[0]+'.csv')
callbacks_list = [csv_logger]

# start training
with tensorflow.device('/device:GPU:0'):
  history = model.fit( train_it, steps_per_epoch=2, validation_data=val_it,
                                  validation_steps=2, epochs=epochs, verbose=2, callbacks = callbacks_list)
# model.fit(train_it, steps_per_epoch=10, validation_data=val_it, validation_steps=1, epochs=epochs,
# verbose=1)
# print(model_name)
# save the model for later use
# model.save( model_name )
# score = model.evaluate(np.array(data), np.array(labels))
# print(score)

TensorFlow is already loaded. Please restart the runtime to change versions.
Tensorflow version 2.1.0-rc1
Found 490 images belonging to 9 classes.
Found 167 images belonging to 9 classes.
batches_in_dataset: 12.25
epochs: 120
train_batch_size: 40
val_batch_size: 40
model_name: /content/models/image_data_blue_light_split_sq350_e120_tb40_vb40_aug-hor-rotran20-briran0_8;1_2).h5
/content/weights-improvement-e{epoch:02d}-va{val_acc:.2f}.h5
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 2 steps, validate for 2 steps
Epoch 1/120
2/2 - 8s - loss: 2.1920 - acc: 0.1200 - val_loss: 2.2048 - val_acc: 0.1125
Epoch 2/120
2/2 - 9s - loss: 2.2000 - acc: 0.1500 - val_loss: 2.2058 - val_acc: 0.1125
Epoch 3/120
2/2 - 9s - loss: 2.1982 - acc: 0.1125 - val_loss: 2.2044 - val_acc: 0.1125
Epoch 4/120
2/2 - 8s - loss: 2.1914 - acc: 0.1000 - val_loss: 2.2032 - val_acc: 0.1125
Epoch 5/120
2/2 - 8s - loss: 2.1860 - acc: 0.1125 - val_loss: 2.2047 - val_acc: 0.1125
Epoch 6/120
2/2 - 8s - loss: 2.1953 

In [0]:

import pandas as pd
df = pd.DataFrame(history.history)
print(df.head())
csv_path = model_name.split('.')[0]+'.csv'
print(csv_path)
df.to_csv(model_name.split('.')[0]+'.csv')

NameError: ignored

In [0]:
sq_model = SqueezeNet( input_shape=(roi_square_size, roi_square_size, 3), include_top=False )
sq_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 350, 350, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 174, 174, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
relu_conv1 (Activation)         (None, 174, 174, 64) 0           conv1[0][0]                      
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 86, 86, 64)   0           relu_conv1[0][0]                 
__________________________________________________________________________________________________
fire2/sque

In [0]:
# verbose=1)
print(model_name)
# save the model for later use
model.save( model_name )
# score = model.evaluate(np.array(data), np.array(labels))
# print(score)

/content/drive/My Drive/ML Mofos SMR/colab_files/models/image_data_all_inbalanced_split_sq350_e465_tb20_vb20_aug-hor-rotran20.h5


In [0]:
from keras.utils import plot_model
plot_model(model, to_file=base_path+'model.png')

In [0]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

In [0]:
# next step is to use TPU's
# https://colab.research.google.com/notebooks/tpu.ipynb#scrollTo=G2pjP0c5VqhK